In [3]:
import sys
import os

# Add the src folder to the system path
sys.path.append(os.path.abspath("../src"))

In [4]:
from dotenv import load_dotenv
from openai import OpenAI
from chatgpt_tools.prompts import translate_word, generate_sentence, dictionarize_word
from chatgpt_tools.tts import get_audio
from anki_tools.anki_deck_creator import insert_or_create_anki_deck

load_dotenv()
client = OpenAI()

In [5]:
def save_into_database(word, translation, sentence, audio_path, dictionary_form):
    print(f"Saving to database: {word}, {translation}, {sentence}, {audio_path}, {dictionary_form}")
    return "Data saved successfully"

def process_word(danish_word, custom_sentence="", language="Danish"):
    translation = translate_word(client, danish_word, source_lang=language)
    sentence = custom_sentence if custom_sentence else generate_sentence(client, danish_word, language)
    dictionary_form = dictionarize_word(client, danish_word, language)
    audio_path = get_audio(client, danish_word, language)
    return translation, sentence, audio_path, dictionary_form


In [6]:
def save_to_database(danish_word, translation, sentence, audio_path, dictionary_form):
    print(f"Audio path: {audio_path}")
    deck_id = 2059400110
    deck_name = "Danish"
    model_id = 1607392319
    model_name = "Danish with Audio"

    cards = [
        {
            "front": f"<div>{danish_word}</div><div>{sentence}</div>",
            "back": f"<div>{translation}</div><div>{dictionary_form}</div>",
            "front_audio_path": audio_path,
        },
        {
            "front": f"<div>{translation}</div>",
            "back": f"<div>{danish_word}</div><div>{sentence}</div><div>{dictionary_form}</div>",
            "back_audio_path": audio_path,
        },
    ]

    output_path = os.path.join(os.getcwd(), "danish_deck.apkg")

    try:
        deck = insert_or_create_anki_deck(deck_id, deck_name, model_id, model_name, cards, output_path)
        print(f"Anki deck operation completed. Deck file: {output_path}")
        print(f"Cards in deck: {len(deck.notes)}")
        return f"Data saved successfully. {len(deck.notes)} cards in Anki deck."
    except Exception as e:
        print(f"Error creating Anki deck: {e}")
        return f"Error creating Anki deck: {e}"


In [7]:
def process_words(word_list):
    results = []
    for word in word_list:
        translation, sentence, audio_path, dictionary_form = process_word(word)
        save_into_database(word, translation, sentence, audio_path, dictionary_form)
        result = save_to_database(word, translation, sentence, audio_path, dictionary_form)
        results.append(result)
    return results


In [8]:
danish_words = [
    "en enke",
    "et billede",
    # "et ægteskab",
    # "Det tror jeg.",
    # "Det håber jeg.",
    # "et menneske",
    # "at tage afsted",
    # "jeg tager aldrig brusebad",
    # "jeg er tit sulten...",
    # "at gøre rent",
    # "at være er var været",
    # "at have har havde haft",
    # "Han bor sammen med sin kone.",
    # "Han bor sammen med hans kone.",
    # "Hendes mand kommer i Budapest.",
    # "et barn børn",
    # "søskende",
    # "bedstefar, bedstemor, bedsteforældre",
    # "Lejlighed",
    # "så tit",
    # "at være gift - skilt",
    # "at læse",
    # "at lære",
    # "at studere",
    # "at savne",
    # "at tænke på",
    # "en nationalitet",
    # "et modersmal",
    # "at være er",
    # "at bo bor",
    # "en sal",
    # "til højre",
    # "til venstre",
    # "et telefonnumer",
    # "at tro - jeg tror",
    # "at synes - jeg synes",
    # "at slappe af"
]

process_results = process_words(danish_words)
print(process_results)

Saving to database: en enke, a widow, En enke sørger stadig over sin afdøde mand., audio_en enke.mp3, enke (substantiv, fælleskøn)
Meaning in English: widow
Bøjning: enken, enker, enkerne
Udtale: [ˈɛŋə]

    flertal: enker
    genitiv: enkes
    Oprindelse: fra gammeldansk enkia, afledt af norrønt enki, som betyder 'én', 'alene'
Audio path: audio_en enke.mp3
Deck file /mnt/d/OneDrive/Projects/anki-words-builder/notebooks/danish_deck.apkg exists, updating it.
Adding 2 cards to the deck...
Processing card 1/2
Processing card 2/2
Updating existing deck at /mnt/d/OneDrive/Projects/anki-words-builder/notebooks/danish_deck.apkg
Total cards in deck: 2
Anki deck operation completed. Deck file: /mnt/d/OneDrive/Projects/anki-words-builder/notebooks/danish_deck.apkg
Error creating Anki deck: 'NoneType' object has no attribute 'notes'
Saving to database: et billede, a picture, Jeg har et billede af min familie på skrivebordet., audio_et billede.mp3, billede (substantiv, intetkøn)
    Betydning på 